In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(input_.loc[i, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_tr_1 = html.xpath('//table[@class="table c-table c-table--split"]/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_th = [th.strip() for th in html.xpath('//table[@class="table c-table c-table--striped"]/thead/tr/th/text()')]

                # = = = = = = = = = = = = = = =
                
                list_tr_2 = html.xpath('//table[@class="table c-table c-table--striped"]/tbody/tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'序号': int(input_.loc[i, 'No.']),
                                        'Url': input_.loc[i, 'Url'],
                                        'Kit_序号': [j+1 for j in range(len(list_tr_2))]})

                for tr in list_tr_1:
                    df_temp[tr.xpath('./th/text()')[0].strip()] = '' if len(tr.xpath('./td/text()')) == 0 else tr.xpath('./td/text()')[0].strip()

                for j in range(len(list_th)):
                    list_td = [tr.xpath('./td')[j] for tr in list_tr_2]
                    list_td = ['' if td.text == None else td.text.strip() for td in list_td]
                    df_temp['Kit_'+list_th[j]] = list_td

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Url': input_.loc[i, 'Url'],
                                     'No.': int(input_.loc[i, 'No.'])}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(str(input_.loc[i, 'No.']) +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Kit_No.'], ascending=[True, True])
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True])
    output_error.to_excel('./part_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：531

12  <->  [ok] - 剩余数量：511 - 当前时间：18:36:39
20  <->  [ok] - 剩余数量：510 - 当前时间：18:36:39
16  <->  [ok] - 剩余数量：509 - 当前时间：18:36:39
3  <->  [ok] - 剩余数量：508 - 当前时间：18:36:40
1  <->  [ok] - 剩余数量：507 - 当前时间：18:36:40
19  <->  [ok] - 剩余数量：506 - 当前时间：18:36:40
6  <->  [ok] - 剩余数量：505 - 当前时间：18:36:40
15  <->  [ok] - 剩余数量：504 - 当前时间：18:36:40
5  <->  [ok] - 剩余数量：503 - 当前时间：18:36:40
4  <->  [ok] - 剩余数量：502 - 当前时间：18:36:41
9  <->  [ok] - 剩余数量：501 - 当前时间：18:36:41
13  <->  [ok] - 剩余数量：500 - 当前时间：18:36:41
10  <->  [ok] - 剩余数量：499 - 当前时间：18:36:41
14  <->  [ok] - 剩余数量：498 - 当前时间：18:36:41
2  <->  [ok] - 剩余数量：497 - 当前时间：18:36:41
22  <->  [ok] - 剩余数量：496 - 当前时间：18:36:41
21  <->  [ok] - 剩余数量：495 - 当前时间：18:36:42
17  <->  [ok] - 剩余数量：494 - 当前时间：18:36:42
24  <->  [ok] - 剩余数量：493 - 当前时间：18:36:42
23  <->  [ok] - 剩余数量：492 - 当前时间：18:36:42
30  <->  [ok] - 剩余数量：491 - 当前时间：18:36:43
27  <->  [ok] - 剩余数量：490 - 当前时间：18:36:43
8  <->  [ok] - 剩余数量：489 - 当前时间：18:36:43
35  <->  [ok] - 剩余数量：488 - 当前时间：18:36:43
26  <->  [ok] -

214  <->  [ok] - 剩余数量：313 - 当前时间：18:37:25
166  <->  [ok] - 剩余数量：312 - 当前时间：18:37:26
215  <->  [ok] - 剩余数量：311 - 当前时间：18:37:26
217  <->  [ok] - 剩余数量：310 - 当前时间：18:37:27
112  <->  [ok] - 剩余数量：309 - 当前时间：18:37:27
218  <->  [ok] - 剩余数量：308 - 当前时间：18:37:27
207  <->  [ok] - 剩余数量：307 - 当前时间：18:37:27
216  <->  [ok] - 剩余数量：306 - 当前时间：18:37:27
221  <->  [ok] - 剩余数量：305 - 当前时间：18:37:28
223  <->  [ok] - 剩余数量：304 - 当前时间：18:37:29
219  <->  [ok] - 剩余数量：303 - 当前时间：18:37:29
220  <->  [ok] - 剩余数量：302 - 当前时间：18:37:29
226  <->  [ok] - 剩余数量：301 - 当前时间：18:37:29
225  <->  [ok] - 剩余数量：300 - 当前时间：18:37:30
222  <->  [ok] - 剩余数量：299 - 当前时间：18:37:31
227  <->  [ok] - 剩余数量：298 - 当前时间：18:37:31
224  <->  [ok] - 剩余数量：297 - 当前时间：18:37:31
228  <->  [ok] - 剩余数量：296 - 当前时间：18:37:31
230  <->  [ok] - 剩余数量：295 - 当前时间：18:37:31
229  <->  [ok] - 剩余数量：294 - 当前时间：18:37:32
211  <->  [ok] - 剩余数量：293 - 当前时间：18:37:32
237  <->  [ok] - 剩余数量：292 - 当前时间：18:37:33
236  <->  [ok] - 剩余数量：291 - 当前时间：18:37:33
238  <->  [ok] - 剩余数量：290 - 当前时间：1

413  <->  [ok] - 剩余数量：116 - 当前时间：18:39:38
416  <->  [ok] - 剩余数量：115 - 当前时间：18:39:40
415  <->  [ok] - 剩余数量：114 - 当前时间：18:39:41
414  <->  [ok] - 剩余数量：113 - 当前时间：18:39:42
417  <->  [ok] - 剩余数量：112 - 当前时间：18:39:42
418  <->  [ok] - 剩余数量：111 - 当前时间：18:39:42
384  <->  [ok] - 剩余数量：110 - 当前时间：18:39:44
419  <->  [ok] - 剩余数量：109 - 当前时间：18:39:44
421  <->  [ok] - 剩余数量：108 - 当前时间：18:39:44
420  <->  [ok] - 剩余数量：107 - 当前时间：18:39:45
425  <->  [ok] - 剩余数量：106 - 当前时间：18:39:46
424  <->  [ok] - 剩余数量：105 - 当前时间：18:39:47
426  <->  [ok] - 剩余数量：104 - 当前时间：18:39:48
59  <->  [ok] - 剩余数量：103 - 当前时间：18:39:48
427  <->  [ok] - 剩余数量：102 - 当前时间：18:39:49
428  <->  [ok] - 剩余数量：101 - 当前时间：18:39:50
429  <->  [ok] - 剩余数量：100 - 当前时间：18:39:50
431  <->  [ok] - 剩余数量：99 - 当前时间：18:39:51
432  <->  [ok] - 剩余数量：98 - 当前时间：18:39:52
430  <->  [ok] - 剩余数量：97 - 当前时间：18:39:52
18  <->  [ok] - 剩余数量：96 - 当前时间：18:39:53
434  <->  [ok] - 剩余数量：95 - 当前时间：18:39:54
433  <->  [ok] - 剩余数量：94 - 当前时间：18:39:55
435  <->  [ok] - 剩余数量：93 - 当前时间：18:39:55
9